In [1]:
import boto3
import pandas as pd


In [2]:
def read_csv_from_s3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """Downloads a csv file from an S3 bucket
    
    Args:
        bucket (str):
            - The bucket where the files at.
        path (str):
            - The folders to the file.
        filename (str):
            - Name of the file.

    Returns:
        pd.DataFrame:
            - A DataFrame of the downloaded file.
    """
    s3 = boto3.client("s3")
    
    full_path = f"{path}{filename}"
    
    object = s3.get_object(Bucket=bucket, Key=full_path)
    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))
    
    return output_df

In [ ]:
s3 = boto3.client("s3")
bucket = "cubix-chicago-taxi-sm"

payment_type_path = "transformed_data/payment_type"
payment_type = read_csv_from_s3(bucket= bucket, path= payment_type_path, "payment_type_master.csv")